<a href="https://colab.research.google.com/github/johntango/DeepSeek-R1-Medical/blob/main/fine_tune_deepseek_r1_math_gsm8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setting Up Working Enviroment

In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-0nup073o
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-0nup073o
  Resolved https://github.com/unslothai/unsloth.git to commit d1d15f1d14f1168837d29b9c08e9b6d63945d469
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.2.12-py3-none-any.whl size=187179 sha256=04009657905af579d309300f1b8a0eb226a230b2784df52fcec2c494e4fd81da
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezjvjtz1/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.2.12
    Uninstalling unsloth-2025.2.12:
      Successfully uninstalled unsloth-2025.2.12


In [ ]:
!pip install --upgrade unsloth
!pip install --upgrade peft
!pip install --upgrade transformers

In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('Lora')
login(hf_token)

In [ ]:
import wandb

wb_token = userdata.get('WANDB_API_KEY')

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jrwtango (jrwtango-massachusetts-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# 2. Loading the Model and Tokenizer

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

# 3. Test the Model with Zero Shot Inference

In [ ]:
style = "medical"
if (style == "medical"):
  prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
  Write a response that appropriately completes the request.
  Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

  ### Instruction:
  You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
  Please answer the following medical question.

  ### Question:
  {}

  ### Response:
  {}"""

else:
  prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
  Write a response that appropriately completes the request.
  Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

  ### Instruction:
  You are a math expert with advanced knowledge in math reasoning.
  Please answer the following math question.

  ### Question:
  {}

  ### Response:
  <think>{}"""

In [ ]:
question = "Amy buy 10 tomatoes at 10 cents each. She sells nearly a third of them. Whats the value of the ones remaining?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


   Amy bought 10 tomatoes at 10 cents each, so the total cost is $1. She sold nearly a third of them, which is about 3 tomatoes. The remaining would be 10 - 3 = 7 tomatoes. Since each tomato costs 10 cents, the value of the remaining tomatoes is 7 * $0.10 = $0.70.
   Amy bought 10 tomatoes at 10 cents each, so the total cost is $1. She sold nearly a third of them, which is about 3 tomatoes. The remaining would be 10 - 3 = 7 tomatoes. Since each tomato costs 10 cents, the value of the remaining tomatoes is 7 * $0.10 = $0.70.
   Amy bought 10 tomatoes at 10 cents each, so the total cost is $1. She sold nearly a third of them, which is about 3 tomatoes. The remaining would be 10 - 3 = 7 tomatoes. Since each tomato costs 10 cents, the value of the remaining tomatoes is 7 * $0.10 = $0.70.
   Amy bought 10 tomatoes at 10 cents each, so the total cost is $1. She sold nearly a third of them, which is about 3 tomatoes. The remaining would be 10 - 3 = 7 tomatoes. Since each tomato costs 10 cent

# 4. Loading and processing the dataset

In [ ]:
# make this for math or medical data
style = "medical"
if (style == "medical"):
    train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
    Write a response that appropriately completes the request.
    Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

    ### Instruction:
    You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
    Please answer the following medical question.

    ### Question:
    {}

    ### Response:
    <think>
    {}
    </think>
    {}"""
else:
    train_prompt_style = """Below is a question that involves math.
    Write a response that appropriately completes the request.
    Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

    ### Instruction:
    You are a math expert with advanced knowledge in math reasoning.
    Please answer the following math question.

    ### Question:
    {}

    ### Response:
    <think>
    {}
    </think>
    {}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
if(style == "medical"):
    def formatting_prompts_func(examples):
        inputs = examples["Question"]
        cots = examples["Complex_CoT"]
        outputs = examples["Response"]
        texts = []
        for input, cot, output in zip(inputs, cots, outputs):
            text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
            texts.append(text)
        return {
            "text": texts,
        }
else:
    def formatting_prompts_func(examples):
        inputs = examples["question"]
        outputs = examples["answer"]
        texts = []
        for input, output in zip(inputs, outputs):
            text = train_prompt_style.format(input, output) + EOS_TOKEN
            texts.append(text)
        return {
            "text": texts,
        }

In [ ]:
from datasets import load_dataset
if(style == "medical"):
    dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
else:
    dataset = load_dataset("openai/gsm8k","main","en", split = "train[0:1000",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context.\n    Write a response that appropriately completes the request.\n    Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n    ### Instruction:\n    You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\n    Please answer the following medical question.\n\n    ### Question:\n    A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n    ### Response:\n    <think>\n    Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever s

# 5. Fine - Tune the LLM

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

AttributeError: 'PeftModelForCausalLM' object has no attribute '_unwrapped_old_generate'

In [ ]:
trainer_stats = trainer.train()

NameError: name 'trainer' is not defined

# 6. Model Inference After Fine-Tuning

In [ ]:
question = "John is 5 years older than his brother who is younger by the number of letters in Italy. How old is the sister?"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>
Okay, let's see. We have a 55-year-old woman with a history of urine leakage whenever she does things like laugh or lift heavy objects. That sounds like a urinary incontinence issue. She's not having any urgency or leakage at night, which is good news because it means she's not dealing with nocturnal enuresis. 

Now, let's think about what might be causing her symptoms. Since she doesn't have urgency, it's unlikely to be a problem with her bladder emptying completely on its own. That points us towards a possible urethral issue, like a urethral sphincter deficiency. 

We've already done a Q-tip test, which helps check for urethral resistance. If the test shows low resistance, it's a strong indicator that we're dealing with urethral sphincter deficiency. This makes me think about how the bladder might function in such a case. 

In urethral sphincter deficiency, the bladder is usually able to empty on its own without any need for detrusor muscle activity. So, during cystometry, I

# 7. Saving the model locally & Hugging Face Hub

In [ ]:
new_model_local = "DeepSeek-R1-Fine-tuned-Medical"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.77 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 38%|███▊      | 12/32 [00:01<00:02,  8.44it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:56<00:00,  3.64s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeek-R1-Fine-tuned-Medical/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Fine-tuned-Medical/pytorch_model-00002-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Fine-tuned-Medical/pytorch_model-00003-of-00004.bin...
Unsloth: Saving DeepSeek-R1-Fine-tuned-Medical/pytorch_model-00004-of-00004.bin...
Done.


In [ ]:
new_model_online = "YoussefHosni/DeepSeek-R1-Fine-tuned-Medical"
model.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

model.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")